<a href="https://colab.research.google.com/github/ynishant77/11785IDL/blob/master/hw1p2_pre.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
files.upload() #upload kaggle.json

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"yashnishant","key":"fd7f210c66481e76e821c41b24b6c51e"}'}

In [ ]:
!pip install -q kaggle

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json

kaggle.json


## Downloading data from Kaggle

In [ ]:
!kaggle competitions download -c 11-785-fall-20-homework-1-part-2

 53% 5.00M/9.45M [00:00<00:00, 28.4MB/s]
100% 9.45M/9.45M [00:00<00:00, 37.3MB/s]
 99% 1.55G/1.57G [00:28<00:00, 64.3MB/s]
100% 1.57G/1.57G [00:28<00:00, 58.3MB/s]
 89% 87.0M/98.1M [00:01<00:00, 35.8MB/s]
100% 98.1M/98.1M [00:02<00:00, 47.1MB/s]
  0% 0.00/3.36M [00:00<?, ?B/s]
100% 3.36M/3.36M [00:00<00:00, 224MB/s]
  0% 0.00/3.19k [00:00<?, ?B/s]
100% 3.19k/3.19k [00:00<00:00, 3.34MB/s]
  0% 0.00/622k [00:00<?, ?B/s]
100% 622k/622k [00:00<00:00, 204MB/s]
 82% 81.0M/98.6M [00:02<00:00, 18.6MB/s]
100% 98.6M/98.6M [00:03<00:00, 34.1MB/s]


In [ ]:
!unzip dev.npy.zip
!unzip dev_labels.npy.zip
!unzip train.npy.zip
!unzip train_labels.npy.zip
!unzip test.npy.zip
#!unzip sample.csv.zip

Archive:  dev.npy.zip
  inflating: dev.npy                 
Archive:  dev_labels.npy.zip
  inflating: dev_labels.npy          
Archive:  train.npy.zip
  inflating: train.npy               
Archive:  train_labels.npy.zip
  inflating: train_labels.npy        
Archive:  test.npy.zip
  inflating: test.npy                


In [ ]:
import numpy as np
import torch
import sys
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils import data
from torch.utils.data import Dataset, DataLoader

import matplotlib.pyplot as plt
import time
from tqdm import tqdm
cuda = torch.cuda.is_available()
cuda

True

## Converting .npy to ndarray

In [ ]:
dev_data = np.load('dev.npy',allow_pickle= True)
dev_labels = np.load('dev_labels.npy',allow_pickle= True)
train_data = np.load('train.npy',allow_pickle= True)
train_labels = np.load('train_labels.npy',allow_pickle= True)
test_data = np.load('test.npy',allow_pickle= True)

## Exploring Data

In [ ]:
print(type(dev_data))
print(dev_data.shape)
print(dev_data[2].shape)

<class 'numpy.ndarray'>
(2332,)
(987, 13)


In [ ]:
print(type(dev_labels))
print(dev_labels.shape)
print(dev_labels[2].shape)

<class 'numpy.ndarray'>
(2332,)
(987,)


In [ ]:
print(type(train_data))
print(train_data.shape)
print(train_data[1].shape)

<class 'numpy.ndarray'>
(22002,)
(1592, 13)


In [ ]:
dev_data[5].shape

(561, 13)

In [ ]:
print(type(test_data))
print(test_data.shape)
print(train_data[0].shape)

<class 'numpy.ndarray'>
(2251,)
(1406, 13)


In [ ]:
test_data_flat = np.vstack(test_data)

In [ ]:
test_data_flat.shape

(1593223, 13)

In [ ]:
train_Data_flat = np.vstack(train_data)

In [ ]:
train_Data_flat.shape

(27329537, 13)

In [ ]:
train_data.shape

(22002,)

In [ ]:
train_first_half = train_data[:11001]
train_first_half.shape

(11001,)

In [ ]:
train_Data_fh_flat = np.vstack(train_first_half)

In [ ]:
train_Data_fh_flat.shape

(13523802, 13)

## Batch Creation

In [ ]:
train_data = np.vstack(train_data)
train_labels = np.concatenate(train_labels)
print(train_data.shape)
print(train_labels.shape)

(27329537, 13)
(27329537,)


In [ ]:
dev_data = np.vstack(dev_data)
dev_labels = np.concatenate(dev_labels)
print(dev_data.shape)
print(dev_labels.shape)

(1598404, 13)
(1598404,)


In [ ]:
test_data = np.vstack(test_data)
print(test_data.shape)

(1593223, 13)


### Padding

In [ ]:
context = 7
train_data =  np.pad(train_data,((context, context), (0, 0)),'constant', constant_values=(0, 0))
train_labels = np.pad(train_labels,(context, context),'constant', constant_values=(0, 0))
print(train_data.shape)
print(train_labels.shape)

(27329551, 13)
(27329551,)


In [ ]:
context = 7
dev_data =  np.pad(dev_data,((context, context), (0, 0)),'constant', constant_values=(0, 0))
dev_labels = np.pad(dev_labels,(context, context),'constant', constant_values=(0, 0))
print(dev_data.shape)
print(dev_labels.shape)

(1598418, 13)
(1598418,)


In [ ]:
context = 7
test_data =  np.pad(test_data,((context, context), (0, 0)),'constant', constant_values=(0, 0))
print(test_data.shape)

(1593237, 13)


### Dataloader

In [ ]:
class Mydataset(Dataset):
  def __init__(self, X, Y):
    self.X = X
    self.Y = Y
    self.context = 7

  def __len__(self):
    return len(self.Y)-2*self.context

  def __getitem__(self,index):
    X = self.X[index:index+2*self.context+1].float().reshape(-1)
    Y = self.Y[index+self.context].long()
    return X, Y

In [ ]:
train_data_tensor = torch.Tensor(train_data)
train_label_tensor = torch.Tensor(train_labels)
print(train_data_tensor.shape)
print(train_label_tensor.shape)

torch.Size([27329551, 13])
torch.Size([27329551])


In [ ]:
dev_data_tensor = torch.Tensor(dev_data)
dev_label_tensor = torch.Tensor(dev_labels)
print(dev_data_tensor.shape)
print(dev_label_tensor.shape)

torch.Size([1598418, 13])
torch.Size([1598418])


In [ ]:
test_data_tensor = torch.Tensor(test_data)
print(test_data_tensor.shape)

torch.Size([1593237, 13])


In [ ]:
del(train_data)
del(train_labels)
del(dev_data)
del(dev_labels)
del(test_data)

In [ ]:
train_dataset = Mydataset(train_data_tensor, train_label_tensor)
val_dataset = Mydataset(dev_data_tensor, dev_label_tensor)

In [ ]:
cuda = torch.cuda.is_available()
num_workers = 2 if cuda else 0
train_loader_args = dict(shuffle=True, batch_size = 1024, num_workers = num_workers, pin_memory = True)if cuda\
                    else dict(shuffle=True, batch_size=256)
train_loader = DataLoader(train_dataset, **train_loader_args)

In [ ]:
val_loader_args = dict(shuffle=False, batch_size=1024, num_workers=num_workers, pin_memory=True) if cuda\
                    else dict(shuffle=False, batch_size=1)
val_loader = DataLoader(val_dataset, **val_loader_args)

In [ ]:
print(len(train_dataset))
print(len(val_dataset))

27329537
1598404


## Model Building

In [ ]:
model = torch.nn.Sequential(
    torch.nn.Linear(195,1024),
    torch.nn.ReLU(),
    torch.nn.Linear(1024,346))

In [ ]:
device = torch.device("cuda" if cuda else "cpu")
model.to(device)
print(model)

Sequential(
  (0): Linear(in_features=195, out_features=1024, bias=True)
  (1): ReLU()
  (2): Linear(in_features=1024, out_features=346, bias=True)
)


In [ ]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [ ]:
def train_epoch(model, train_loader, criterion, optimizer):
    model.train()

    running_loss = 0.0
    
    start_time = time.time()
    for batch_idx, (data, target) in enumerate(train_loader):   
        optimizer.zero_grad()   # .backward() accumulates gradients
        data = data.to(device)
        target = target.to(device) # all data & model on same device

        outputs = model(data)
        loss = criterion(outputs, target)
        running_loss += loss.item()

        loss.backward()
        optimizer.step()
    
    end_time = time.time()
    
    running_loss /= len(train_loader)
    print('Training Loss: ', running_loss, 'Time: ',end_time - start_time, 's')
    return running_loss

In [ ]:
def test_model(model, test_loader, criterion):
    with torch.no_grad():
        model.eval()

        running_loss = 0.0
        total_predictions = 0.0
        correct_predictions = 0.0

        for batch_idx, (data, target) in enumerate(test_loader):   
            data = data.to(device)
            target = target.to(device)

            outputs = model(data)

            _, predicted = torch.max(outputs.data, 1)
            total_predictions += target.size(0)
            correct_predictions += (predicted == target).sum().item()

            loss = criterion(outputs, target).detach()
            running_loss += loss.item()


        running_loss /= len(test_loader)
        acc = (correct_predictions/total_predictions)*100.0
        print('Testing Loss: ', running_loss)
        print('Testing Accuracy: ', acc, '%')
        return running_loss, acc

In [ ]:
n_epochs = 5
Train_loss = []
Val_loss = []
Val_acc = []

for i in range(n_epochs):
    train_loss = train_epoch(model, train_loader, criterion, optimizer)
    val_loss, val_acc = test_model(model, val_loader, criterion)
    Train_loss.append(train_loss)
    Val_loss.append(val_loss)
    Val_acc.append(val_acc)
    print('='*20)

Training Loss:  1.860630058934064 Time:  383.34387493133545 s
Testing Loss:  1.7293539907518372
Testing Accuracy:  51.58013868834163 %
Training Loss:  1.5856943830484842 Time:  369.24140787124634 s
Testing Loss:  1.63971958711772
Testing Accuracy:  53.55148010140115 %
Training Loss:  1.5252284048635998 Time:  367.78746914863586 s
Testing Loss:  1.596282973188685
Testing Accuracy:  54.648073953768886 %
Training Loss:  1.494009638109561 Time:  370.6776371002197 s
Testing Loss:  1.5833762075410753
Testing Accuracy:  54.9756507115848 %
Training Loss:  1.4746513528027039 Time:  366.5221130847931 s
Testing Loss:  1.5645936137968572
Testing Accuracy:  55.30823246188072 %


In [ ]:
print("hello")

hello


In [ ]:
torch.save(model.state_dict(),'test.t1')

## Testing

In [ ]:
device = torch.device("cuda" if cuda else "cpu")
model = torch.nn.Sequential(
    torch.nn.Linear(195,1024),
    torch.nn.ReLU(),
    torch.nn.Linear(1024,346))

In [ ]:
model.load_state_dict(torch.load('test.t1'))
model.to(device)

Sequential(
  (0): Linear(in_features=195, out_features=1024, bias=True)
  (1): ReLU()
  (2): Linear(in_features=1024, out_features=346, bias=True)
)

In [ ]:
class MyTestdataset(Dataset):
  def __init__(self, X):
    self.X = X
    self.context = 7

  def __len__(self):
    return len(self.X)-2*self.context

  def __getitem__(self,index):
    X = self.X[index:index+2*self.context+1].float().reshape(-1)
    return X

In [ ]:
num_workers = 2
test_dataset = MyTestdataset(test_data_tensor)
test_loader_args = dict(shuffle=False, batch_size=1024, num_workers=num_workers, pin_memory=True) if cuda\
                    else dict(shuffle=False, batch_size=1)
test_loader = DataLoader(test_dataset, **test_loader_args)

In [ ]:
len(test_dataset)

1593223

In [ ]:
#del(target)
torch.cuda.empty_cache()

In [ ]:
result = []
with torch.no_grad():
  for test_data in test_loader:
    data = test_data.to(device)
    outputs = model(data)
    _, predicted = torch.max(outputs.data, 1)
    result.append(predicted)

In [ ]:
len(result)

1556

In [ ]:
test_target = torch.cat(result, dim=0)

In [ ]:
test_result = test_target.data.cpu().numpy()

In [ ]:
test_result[0]

1

In [ ]:
import csv
with open('submission1.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["id", "label"])
    for idx,data in enumerate(test_result):
      writer.writerow([idx, data])